# Berman Carbonate Database 
Required system packages and initialization

In [21]:
import pandas as pd
import numpy as np
import sympy as sym
import os
import thermoengine
import dill

import importlib
sym.init_printing()



# Create coder module for pure phase using Berman Std State
* Head over to Berman-std-state-model Notebook to see coder expressions and model formulation

In [22]:
%run core.ipynb
%run Berman-std-state-model.ipynb

In [23]:
def Berman_params(Phase='Name', Formula='', 
                  H_TrPr=-1e6, S_TrPr=1e2, V_TrPr=3,
                  k0=1e2, k1=-10.0E2, k2=-5.0E5, k3=16.667E7,
                  v1=-1.0E-6, v2=0.0, v3=9.0E-6, v4=300.0E-10):
    
    param_dict = {
        'Phase': Phase, 'Formula': Formula,
        'T_r': 298.15, 'P_r': 1.0,
        'H_TrPr': H_TrPr, 'S_TrPr': S_TrPr, 'V_TrPr': V_TrPr,
        'k0': k0, 'k1': k1, 'k2': k2, 'k3': k3,
        'v1': v1, 'v2': v2, 'v3': v3, 'v4': v4,
        'l1': 0.0, 'l2': 0.0, 'k_lambda': 0.0,
        'T_lambda_Pr': 0.0, 'T_lambda_ref': 0.0, 'H_t': 0.0,
        'd0': 0.0, 'd1': 0.0, 'd2': 0.0, 'd3': 0.0, 'd4': 0.0, 'd5': 0.0,
        'T_D': 0.0, 'T_D_ref': 0.0}
    return param_dict

# Build up each phase from known properties
* taken from Berman 1988 for calcite, magnesite, dolomite
* other phases invented based on Robie (1995), Berman and Brown (1985), and NIST constraints

In [24]:
phase_params = {}

In [25]:
lookup_phase_info('calcite')

Abbrev     Name Formula Parent
33    Cal  Calcite   CaCO3    NaN

In [26]:
phase_params['Cal'] = Berman_params(
    Phase='Calcite', Formula='Ca(1)C(1)O(3)',
    H_TrPr=-1206819.0, S_TrPr=91.725, V_TrPr=3.690,
    k0=178.19, k1=-16.577E2, k2=-4.827E5, k3=16.660E7,
    v1=-1.4E-6, v2=0.0, v3=8.907E-6, v4=227.402E-10)

In [27]:
lookup_phase_info('magnesite')

Abbrev       Name Formula Parent
123    Mgs  Magnesite   MgCO3    NaN
124    Mag  Magnetite   Fe3O4    NaN

In [28]:
phase_params['Mgs'] = Berman_params(
    Phase='Magnesite', Formula='Mg(1)C(1)O(3)',
    H_TrPr=-1113636.0, S_TrPr=65.210, V_TrPr=2.803,
    k0=162.30, k1=-11.093E2, k2=-48.826E5, k3=87.466E7,
    v1=-0.890E-6, v2=2.212E-12, v3=18.436E-6, v4=415.968E-10)

In [29]:
lookup_phase_info('aragonite')

Abbrev        Name           Formula Parent
25     Arg   Aragonite             CaCO3    NaN
149     Pg  Paragonite  NaAl3Si3O10(OH)2    NaN

In [30]:
phase_params['Arg'] = Berman_params(
    Phase='Aragonite', Formula='Ca(1)C(1)O(3)',
    H_TrPr=-1206819.0+1100.0, S_TrPr=88.0, V_TrPr=3.415,
    k0=166.62, k1=-14.994E2, k2=0.0, k3=5.449E7,
    v1=-1.4E-6, v2=0.0, v3=8.907E-6, v4=227.402E-10)

In [31]:
lookup_phase_info('siderite')

Abbrev      Name Formula Parent
184     Sd  Siderite   FeCO3    NaN

In [32]:
phase_params['Sd'] = Berman_params(
    Phase='Siderite', Formula='Fe(1)C(1)O(3)',
    H_TrPr=-755900.0, S_TrPr=95.5, V_TrPr=2.938,
    k0=177.36, k1=-16.694E2, k2=-3.551E5, k3=15.078E7,
    v1=-0.890E-6, v2=2.212E-12, v3=18.436E-6, v4=415.968E-10)

In [33]:
lookup_phase_info('dolomite')

Abbrev      Name     Formula Parent
58    Dol  Dolomite  CaMg(CO3)2    NaN

In [34]:
phase_params['Dol'] = Berman_params(
    Phase='Dolomite', Formula='Ca(1)Mg(1)C(2)O(6)',
    H_TrPr=-2324500.0+1100.0, S_TrPr=155.2, V_TrPr=6.434,
    k0=368.02, k1=-37.508E2, k2=0.0, k3=18.079E7,
    v1=-1.4E-6, v2=0.0, v3=8.907E-6, v4=227.402E-10)

# Build Databse

In [35]:
mod_name = 'berman'

In [36]:
modelDB, coder_mod = make_custom_database(mod_name, phase_params)

/Users/aswolf/Documents/projects/ENKI/ThermoEngine/Notebooks/Carbonated-Mantle/working
Creating (once only) generic fast model code file string
Creating (once only) generic model calib code template include file string
Creating (once only) generic model calib code template code file string
Creating (once only) generic calib model code file string
Creating include file ...
... done!
Creating code file ...
... done
Writing include file to working directory ...
Writing code file to working directory ...
Writing pyxbld file to working directory ...
writing pyx file to working directory ...
Compiling code and Python bindings ...
Success! Import the module named  berman
Creating include file ...
... done!
Creating code file ...
... done
Writing include file to working directory ...
Writing code file to working directory ...
Writing pyxbld file to working directory ...
writing pyx file to working directory ...
Compiling code and Python bindings ...
Success! Import the module named  berman
Cre

# Examine Phases in DB
* note Water and liq (melt) are always present according to MELTS and DEW models...

In [37]:
modelDB.phase_info

abbrev phase_name   formula phase_type endmember_num
0    Liq     Liquid             solution            15
1    Cal    Calcite     CaCO3       pure             1
2    Mgs  Magnesite     MgCO3       pure             1
3    Arg  Aragonite     CaCO3       pure             1
4     Sd   Siderite     FeCO3       pure             1
5    Dol   Dolomite  CaMgC2O6       pure             1
6    H2O      Water       H2O       pure             1

# Save output phases for later

In [38]:
output = {}

for phsnm in phase_params:
    iphs = modelDB.get_phase(phsnm)
    output[phsnm] = iphs
    

In [39]:
output

{'Arg': <thermoengine.phases.PurePhase at 0x127da20f0>,
 'Cal': <thermoengine.phases.PurePhase at 0x127dbd748>,
 'Dol': <thermoengine.phases.PurePhase at 0x127dab400>,
 'Mgs': <thermoengine.phases.PurePhase at 0x127dbd6d8>,
 'Sd': <thermoengine.phases.PurePhase at 0x127da2a58>}

In [40]:
with open('carbon-phases.pkl','wb') as file:
    dill.dump(output, file)
    file.close()